# TMDD Benchmarking
## Data generation

This notebook generates a reference data set for benchmarking SAEM implementations with ODE models.

The model is a target-mediated drug disposition (TMDD) model, described for instance in [Dua et al. (2015)](https://doi.org/10.1002/psp4.41), that contains 3 ODEs:

```math
\begin{align*}
\frac{dL}{dt} =& -k_{eL} L -k_{on} LR + k_{off} P, \\
\frac{dR}{dt} =& k_{syn} - k_{def} R -k_{on} LR +k_{off} P, \\
\frac{dP}{dt} =& k_{on} LR + k_{off} P - k_{eP} P, \\
&L(0) = L_0, R(0) = R_0 = \frac{k_{syn}}{k_{deg}}, P(0) = 0, \\
&\frac{k_{off}}{k_{on}} = K_D
\end{align*}
```

## Reference parameter values
The parameters' reference values and distributions are gathered in the table below. Distributed parameters follow a log-normal distribution.

| Parameter | Description | Assumption | Reference value | Spread (log) |
| --- | --- | --- | ---| ---|
| $`k_{off}`$ | Unbinding constant rate | Known | $`1 h^{-1}`$ | None |
| $`k_{D}`$  | Dissociation constant | Known | $`1 nM`$ |  None |
| $`k_{on}`$  | On-rate constant | Known | $`\frac{k_{off}}{k_D}`$ |  None |
| $`inj`$ | Administered concentration | Known | $`10 nanomole`$ | None |
| $`V_c`$ | Distribution volume | Unknown | $`3L`$ | $`\omega^2 = 0.1`$ |
| $`L_0`$ | Initial antibody concentration | Known | $`10 nM`$ | None |
| $`k_{deg}`$  | Receptor elimination rate | Known | $`1e-2 h^{-1}`$ | None |
| $`R_0`$  | Total receptor concentration | Unknown | $`3 nM`$ | $` \omega^2 = 0.1 `$ |
| $`k_{syn}`$  | Receptor synthesis rate | Known | $`R_0 \times k_{deg}`$ |  None |
| $`k_{eL}`$  | Free antibody elimination rate | Unknown | $`5e-2 h^{-1}`$ | $`\omega^2 = 0.5`$ |
| $`k_{eP}`$  | Bound antibody elimination rate | Unknown | $`1e-1 h^{-1}`$ | $` \omega^2 = 0.1 `$ |

In [ ]:
import numpy as np
import pandas as pd
import uuid

from vpop_calibration import *


%load_ext autoreload
%autoreload 2

In [ ]:
# Define the parameters values
true_R_0 = 3.0  # nM
true_inj = 10  # nanomol
true_Vc = 3  # L
true_k_off = 1.0  # 1 / h
true_k_D = 1.0  # nM
true_k_deg = 1e-2  # 1 / h
true_k_eL = 5e-2  # 1 / h
true_k_eP = 1e-1  # 1 / h


# Define the ODE model
def equations(t, y, k_eL, k_eP, inj, Vc, R0):
    """TMDD model equations.

    Concentrations are expressed in `nM` and time in `h`.

    """
    yL, yR, yP = y
    k_on = true_k_off / true_k_D
    k_syn = true_R_0 * true_k_deg
    dyL = -k_eL * yL - k_on * yL * yR + true_k_off * yP
    dyR = k_syn - true_k_deg * yR - k_on * yL * yR + true_k_off * yP
    dyP = k_on * yL * yR - true_k_off * yP - k_eP * yP
    ydot = [dyL, dyR, dyP]
    return ydot


def init_assignment(k_eL, k_eP, inj, Vc, R0):
    return [inj / Vc, R0, 0.0]


variable_names = ["L", "R", "P"]
parameter_names = ["k_eL", "k_eP", "inj", "Vc", "R0"]

tmdd_model = OdeModel(equations, init_assignment, variable_names, parameter_names)


# NLME model parameters
true_log_MI = {}
true_log_PDU = {
    "k_eL": {"mean": np.log(true_k_eL), "sd": 0.5},
    "k_eP": {"mean": np.log(true_k_eL), "sd": 0.1},
    "R0": {"mean": np.log(true_R_0), "sd": 0.1},
    "Vc": {"mean": np.log(true_Vc), "sd": 0.1},
}

In [ ]:
tmax = 10.0  # hours
nb_steps = 20
time_steps = np.linspace(0.0, tmax, nb_steps).tolist()

error_model_type = "proportional"
true_res_var = [0.02, 0.05, 0.05]

# Define the patients data frame
nb_patients = 100
# Give them a unique id
patients_df = pd.DataFrame({"id": [str(uuid.uuid4()) for _ in range(nb_patients)]})
# Initialize some random properties
rng = np.random.default_rng()
patients_df["protocol_arm"] = "identity"
protocol_design = pd.DataFrame({"protocol_arm": ["identity"], "inj": [true_inj]})

display(patients_df.head())

obs_df = simulate_dataset_from_omega(
    tmdd_model,
    protocol_design,
    time_steps,
    true_log_MI,
    true_log_PDU,
    error_model_type,
    true_res_var,
    None,
    patients_df,
)

display(obs_df.head())

In [ ]:
# Plot the full data set
from plotnine import *

(
    ggplot(obs_df, aes(x="time", y="value", color="id"))
    + facet_grid(cols="output_name")
    + geom_line()
    + geom_point()
    + theme_light()
    + theme(legend_position="none")
)

In [ ]:
# Filter and format the data set for exporting

# Only the antibody concentration is considered
final_obs_df = obs_df.loc[obs_df["output_name"] == "L"][
    ["id", "time", "protocol_arm", "output_name", "value"]
]

display(final_obs_df)

final_obs_df.to_csv(
    "./tmdd_synthetic_data_pysaem.csv", float_format="%.3f", index=False
)

df_saemix = final_obs_df[["id", "time", "value"]]
df_saemix.to_csv("./tmdd_synthetic_data_saemix.csv", float_format="%.3f", index=False)

df_nlmixr2 = final_obs_df[["id", "time", "value"]].rename(
    columns={"value": "DV", "time": "TIME", "id": "ID"}
)
df_nlmixr2["EVID"] = 0
df_nlmixr2["CMT"] = 1
df_nlmixr2["AMT"] = 0
df_nlmixr2.loc[df_nlmixr2["TIME"] == 0, "AMT"] = true_inj
df_nlmixr2.loc[df_nlmixr2["TIME"] == 0, "EVID"] = 1
df_nlmixr2.to_csv(
    "./tmdd_synthetic_data_nlmixr2.csv", sep=" ", float_format="%.3f", index=False
)